# 사전 구축된 ReAct 에이전트에 사용자 지정 시스템 프롬프트 추가하는 방법

<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 가이드는 다음에 대한 이해를 가정합니다:
        <ul>
            <li>            
                <a href="https://python.langchain.com/docs/concepts/messages/#systemmessage">
                    SystemMessage
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/">
                    에이전트 아키텍처
                </a>                   
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/chat_models/">
                    채팅 모델
                </a>
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/tools/">
                    도구
                </a>
            </li>
        </ul>
    </p>
</div> 

이 튜토리얼에서는 [사전 구축된 ReAct 에이전트](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent)에 사용자 지정 시스템 프롬프트를 추가하는 방법을 보여줍니다. 사전 구축된 ReAct 에이전트를 시작하는 방법에 대해서는 [이 튜토리얼](../create-react-agent)을 참조하세요.

문자열을 `prompt` 매개변수로 전달하여 사용자 지정 시스템 프롬프트를 추가할 수 있습니다.


## 설정

먼저, 필요한 패키지를 설치하고 API 키를 설정해 보겠습니다.


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위해 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 파악하고 성능을 향상시킬 수 있습니다. LangSmith를 사용하면 LangGraph로 구축된 LLM 앱을 디버그하고, 테스트하고, 모니터링하기 위한 추적 데이터를 활용할 수 있습니다 — 시작하는 방법에 대해 더 읽어보려면 <a href="https://docs.smith.langchain.com">여기</a>를 클릭하세요.
    </p>
</div>


## 코드


In [1]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)


# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from typing import Literal

from langchain_core.tools import tool


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]

# We can add our system prompt here

prompt = "Respond in Italian"

# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools, prompt=prompt)


## 사용법


In [2]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


In [4]:
inputs = {"messages": [("user", "What's the weather in NYC?")]}

print_stream(graph.stream(inputs, stream_mode="values"))


================================ Human Message =================================

What's the weather in NYC?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_b02uzBRrIm2uciJa8zDXCDxT)
 Call ID: call_b02uzBRrIm2uciJa8zDXCDxT
  Args:
    city: nyc
================================= Tool Message =================================
Name: get_weather

It might be cloudy in nyc
================================== Ai Message ==================================

A New York potrebbe essere nuvoloso.
